## Exercise 1


In [0]:
import pandas as pd
df1 = pd.read_stata('http://www.stata-press.com/data/r11/auto.dta')
print(df1.shape)
df1.head()


(74, 12)


,make,price,mpg,rep78,headroom,trunk,weight,length,turn,displacement,gear_ratio,foreign
0,AMC Concord,4099,22,3.0,2.5,11,2930,186,40,121,3.58,Domestic
1,AMC Pacer,4749,17,3.0,3.0,11,3350,173,40,258,2.53,Domestic
2,AMC Spirit,3799,22,NaN,3.0,12,2640,168,35,121,3.08,Domestic
3,Buick Century,4816,20,3.0,4.5,16,3250,196,40,196,2.93,Domestic
4,Buick Electra,7827,15,4.0,4.0,20,4080,222,43,350,2.41,Domestic


In [0]:
len(df1)

74

In [0]:
df1[5:9]

,make,price,mpg,rep78,headroom,trunk,weight,length,turn,displacement,gear_ratio,foreign
5,Buick LeSabre,5788,18,3.0,4.0,21,3670,218,43,231,2.73,Domestic
6,Buick Opel,4453,26,NaN,3.0,10,2230,170,34,304,2.87,Domestic
7,Buick Regal,5189,20,3.0,2.0,16,3280,200,42,196,2.93,Domestic
8,Buick Riviera,10372,16,3.0,3.5,17,3880,207,43,231,2.93,Domestic


In [0]:
df1.rename(columns={'rep78':'Repair Record 1978'},inplace=True)
df2=df1.groupby(['Repair Record 1978'],as_index=False)['make'].count()
df2.columns=['Repair Record 1978','Freq.']
df2


,Repair Record 1978,Freq.
0,1.0,2
1,2.0,8
2,3.0,30
3,4.0,18
4,5.0,11


In [0]:
df3=df1.groupby(['Repair Record 1978']).agg({'mpg':['count','mean','median'],'price':'max','length':'min'})
df3.columns=['N(mpg)','mean(mpg)','med(mpg)','max(price)','min(length)']
df3

,N(mpg),mean(mpg),med(mpg),max(price),min(length)
Repair Record 1978,,,,,
1.0,2,21.000000,21.0,4934,180
2.0,8,19.125000,18.0,14500,179
3.0,30,19.433333,19.0,15906,142
4.0,18,21.666667,22.5,9735,147
5.0,11,27.363636,30.0,11995,155


In [0]:
df4=df1.groupby(['Repair Record 1978'])['foreign'].value_counts().unstack().fillna(0)
df4['Total']=df4.sum(axis=1)
sums = df4.sum().rename('Total')
df4.append(sums)

foreign,Domestic,Foreign,Total
Repair Record 1978,,,
1.0,2.0,0.0,2.0
2.0,8.0,0.0,8.0
3.0,27.0,3.0,30.0
4.0,9.0,9.0,18.0
5.0,2.0,9.0,11.0
Total,48.0,21.0,69.0


In [0]:
df5=pd.DataFrame(df1.groupby(['Repair Record 1978','foreign'],as_index=False)['mpg'].mean())
df5.pivot(index='Repair Record 1978', columns='foreign', values='mpg')
           

foreign,Domestic,Foreign
Repair Record 1978,,
1.0,21.000000,NaN
2.0,19.125000,NaN
3.0,19.000000,23.333333
4.0,18.444444,24.888889
5.0,32.000000,26.333333


## Exercise 2

In [0]:
df1=pd.read_stata('http://www.stata-press.com/data/r11/reshape1.dta')
df1

,id,sex,inc80,inc81,inc82,ue80,ue81,ue82
0,1.0,0.0,5000.0,5500.0,6000.0,0.0,1.0,0.0
1,2.0,1.0,2000.0,2200.0,3300.0,1.0,0.0,0.0
2,3.0,0.0,3000.0,2000.0,1000.0,0.0,0.0,1.0


In [0]:
df2=pd.wide_to_long(df1, stubnames=['inc','ue'], i=['id'], j='year')
df2.sort_values(by=['id','year','ue'], axis=0)


sex     inc   ue
id  year                  
1.0 80    0.0  5000.0  0.0
    81    0.0  5500.0  1.0
    82    0.0  6000.0  0.0
2.0 80    1.0  2000.0  1.0
    81    1.0  2200.0  0.0
    82    1.0  3300.0  0.0
3.0 80    0.0  3000.0  0.0
    81    0.0  2000.0  0.0
    82    0.0  1000.0  1.0

## Exercise 3

In [0]:
df1=pd.read_stata('http://www.stata-press.com/data/r11/auto.dta')
df1.head()

,make,price,mpg,rep78,headroom,trunk,weight,length,turn,displacement,gear_ratio,foreign
0,AMC Concord,4099,22,3.0,2.5,11,2930,186,40,121,3.58,Domestic
1,AMC Pacer,4749,17,3.0,3.0,11,3350,173,40,258,2.53,Domestic
2,AMC Spirit,3799,22,NaN,3.0,12,2640,168,35,121,3.08,Domestic
3,Buick Century,4816,20,3.0,4.5,16,3250,196,40,196,2.93,Domestic
4,Buick Electra,7827,15,4.0,4.0,20,4080,222,43,350,2.41,Domestic


In [0]:
#https://www.stata.com/manuals13/u25.pdf
df=df1[['rep78','foreign','mpg']]
cat_columns=['rep78','foreign']
for column in cat_columns:
  dummyCols=pd.get_dummies(df[column])
  df=df.join(dummyCols)
  del df[column]

In [0]:
X=df.drop(['mpg'],axis=1)
Y=df1['mpg']

import statsmodels.api as sm
model2=sm.OLS(Y,X).fit()
model2.summary()

# from statsmodels.stats.anova import anova_lm
# anova_results = anova_lm(model)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.249
Model:                            OLS   Adj. R-squared:                  0.182
Method:                 Least Squares   F-statistic:                     3.710
Date:                Wed, 14 Aug 2019   Prob (F-statistic):            0.00304
Time:                        06:25:34   Log-Likelihood:                -223.78
No. Observations:                  74   AIC:                             461.6
Df Residuals:                      67   BIC:                             477.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
1.0            0.0987      4.390      0.022      0.982      -8.665       8.862
2.0           -1.7763      3.002     -0.592      0.556      -7.768       4.215
3.0           -1.7173      2.533     -0.678      0.500      -6.773       3.338
4.0           -0.4814      2.693     -0.179      0.859      -5.856       4.894
5.0            4.4223      3.008      1.470      0.146      -1.582      10.427
Domestic      20.9013      2.364      8.842      0.000      16.183      25.620
Foreign       23.3947      2.701      8.663      0.000      18.004      28.785
==============================================================================
Omnibus:                        0.089   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.957   Jarque-Bera (JB):                0.009
Skew:                          -0.019   Prob(JB):                        0.996
Kurtosis:                       2.963   Cond. No.                         10.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""